In [ ]:
import cv2
import numpy as np
import time
from tkinter import *
from tkinter import filedialog
import eel

                                                                 # Using tkinter to create a mini front-end
#r.title('Object Detection')

eel.init('web')

@eel.expose
def op():
    
    r=Tk()
    
    filename=filedialog.askopenfilename()
    r.destroy()
    yolo(filename)
    
    

@eel.expose
def cam():
    
    filename=0
    yolo(filename)
    
    

@eel.expose
def yolo(filename):
    net = cv2.dnn.readNet("Weight/dhg/yolov2-tiny.weights", "cfg/yolov2-tiny.cfg")   #Lodaing the Yolo model
    classes = []


    with open("Classes/classes.names", "r") as f:                                      #loding the names of classes from the file
        classes = [line.strip() for line in f.readlines()]


    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    
    cap = cv2.VideoCapture(filename)

    

    font = cv2.FONT_HERSHEY_PLAIN
    starting_time = time.time()

    frame_id = 0

    while True:
        _, frame = cap.read()
        frame_id += 1

        height, width, channels = frame.shape


        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        net.setInput(blob)                                                  #Giving the blob as input to the yolo
        outs = net.forward(output_layers)                                                              


        class_ids = []                                                      #Empty list to store the ID of the identified classes of objects
        confidences = []                                                    #Empty list to store the confidence score of each prediction made
        boxes = []                                                          #Empty list to store the co-ordinates of all the objects in the image


        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.2:                                        #If confidence is greater than 0.2(i.e 20%) we concider it a valid prediction 

                    center_x = int(detection[0] * width)                    #Getting the co-ordinates of all the valid predictions made.
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)


                    x = int(center_x - w / 2)                               #Converting the above to rectangular co-ordinates
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])                              #Saving the Co-ordinates.
                    confidences.append(float(confidence))
                    class_ids.append(class_id)


        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)


        for i in range(len(boxes)):

            if i in indexes:

                x, y, w, h = boxes[i]                                                                  
                label = str(classes[class_ids[i]])                                                     
                confidence = confidences[i]
                color = colors[class_ids[i]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)      #Drawing the reactangle 
                cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 20), font, 1.5, color, 2)       #Showing the name of class and the confidence score



        elapsed_time = time.time() - starting_time                          
        fps = frame_id / elapsed_time                                       #Calculating the fps

        cv2.rectangle(frame, (0, 0), (110, 35), (0,0,0), -1)
        cv2.putText(frame, "FPS: " + str(round(fps, 2)), (5, 15), font, 0.7, (255,255,255), 1)
        cv2.putText(frame, "Press 'q' to exit", (5, 29), font, 0.7, (255,255,255), 1)

        cv2.imshow("Object Detection", frame)                                          

        if cv2.waitKey(1) & 0xFF == ord('q'):                               #Press q to exit
              break

    cap.release()
    cv2.destroyAllWindows()

eel.start('index.html', size=(1000, 600))